In [1]:
from faker import Faker
import pandas as pd
import random
import dash
import datetime
import re
import plotly.graph_objs as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Configurar o pandas para exibir todas as linhas e colunas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Criando Tabelas com dados fakes

## Tabelas: Professor, Aluno e Disciplina

In [ ]:
fake = Faker()

# Criando a tabela Professor
num_professores = 5
professor_data = {
    'id': range(1, num_professores + 1),
    'nome': [fake.name() for _ in range(num_professores)],
    'email': [fake.email() for _ in range(num_professores)]
}
professor_df = pd.DataFrame(professor_data)

# Criando a tabela Aluno
num_alunos = 20
aluno_data = {
    'id': range(1, num_alunos + 1),
    'nome': [fake.name() for _ in range(num_alunos)]
}
aluno_df = pd.DataFrame(aluno_data)

# Criando a tabela Disciplina
num_dis = 5
disciplina_data = {
    'id': range(1, num_dis + 1),
    'codigo': ['MAT', 'FIS', 'BIO', 'HIS', 'QUI'],
    'nome': ['Matemática', 'Física', 'Biologia', 'História', 'Química']
}
disciplina_df = pd.DataFrame(disciplina_data)


## Tabela: Turma

In [ ]:
fake = Faker()

# Criando a tabela Turma
turma_data = []
id = 1

# Preencher os dados da turma para cada professor
for index, row in professor_df.iterrows():
    professor_id = row['id']
    disciplina_id = index + 1

    # Selecionar aleatoriamente 10 alunos para cada disciplina
    alunos_ids = random.sample(range(1, num_alunos + 1), 10)
    status = random.choice(['Em andamento', 'Notas pendentes', 'Finalizada'])  # Variando o status
    
    for aluno_id in alunos_ids:
        turma_id = int(f"{professor_id}{disciplina_id}")
        
        # Primeira prova (entre março e abril)
        turma_data.append({
            'id': id,
            'prof_disc_id': turma_id,
            'professor_id': professor_id,
            'disciplina_id': disciplina_id,
            'aluno_id': aluno_id,
            'started_at': fake.date_time_between_dates(datetime_start=pd.to_datetime('2023-03-01'), 
                                                       datetime_end=pd.to_datetime('2023-04-30')).strftime('%Y-%m-%d'),
            'tipo': 'Semestral',
            'status': 'Finalizada',
            'logica_nota_final': '(P1+P2)/2',
            'media_final_minima': 5.0
        })
        id += 1

        # Segunda prova (entre maio e junho) e variamos o status
        turma_data.append({
            'id': id,
            'prof_disc_id': turma_id,
            'professor_id': professor_id,
            'disciplina_id': disciplina_id,
            'aluno_id': aluno_id,
            'started_at': fake.date_time_between_dates(datetime_start=pd.to_datetime('2023-05-01'), 
                                                       datetime_end=pd.to_datetime('2023-06-30')).strftime('%Y-%m-%d'),
            'tipo': 'Semestral',
            'status': status,
            'logica_nota_final': '(P1+P2)/2',
            'media_final_minima': 5.0
        })
        id += 1

turma_df = pd.DataFrame(turma_data)

## Tabela: Nota

In [ ]:
# Filtrar os dados da tabela turma_df para incluir apenas os casos em que o status é "Finalizada"
turma_finalizada = turma_df[turma_df['status'] == 'Finalizada']

# Criando a tabela Notas
notas_data = []

for index, row in turma_finalizada.iterrows():
    turma_id = row['id']
    aluno_id = row['aluno_id']
    started_at = datetime.datetime.strptime(row['started_at'], '%Y-%m-%d')
    
    # Verificar o tipo de nota com base no started_at
    if started_at.month in [3, 4]:
        tipo_nota = 'P1'
    elif started_at.month in [5, 6]:
        tipo_nota = 'P2'
    else:
        continue  # Ignorar se o started_at estiver fora dos períodos desejados
    
    # Gerar notas para cada tipo (P1, P2)
    nota = round(random.uniform(1, 10), 1)  # Gerando nota aleatória entre 1 e 10
    notas_data.append({
        'turma_id': turma_id,
        'aluno_id': aluno_id,
        'nota': nota,
        'aval': tipo_nota
    })

notas_df = pd.DataFrame(notas_data)


## Tabela: Media_Final

In [ ]:
# Criando a tabela de notas finais por disciplina
media_final_data = []

# Iterar sobre cada disciplina na tabela de turma
for disciplina_id in turma_df['disciplina_id'].unique():
    # Filtrar as notas correspondentes à disciplina na tabela de notas
    notas_disciplina = notas_df.merge(turma_df, left_on=['turma_id','aluno_id'], right_on=['id','aluno_id'])
    notas_disciplina = notas_disciplina[(notas_disciplina['disciplina_id'] == disciplina_id)]

    # Iterar sobre cada aluno na disciplina
    for aluno_id in notas_disciplina['aluno_id'].unique():
        # Obter a lógica de notas finais para a disciplina atual
        logica_nota_final = turma_df.loc[turma_df['disciplina_id'] == disciplina_id, 'logica_nota_final'].iloc[0]
        media_final_minima = turma_df.loc[turma_df['disciplina_id'] == disciplina_id, 'media_final_minima'].iloc[0]

        # Usar expressão regular para encontrar todas as variáveis na lógica
        variaveis = re.findall(r'\b\w+\b', logica_nota_final)
        variaveis = [var for var in variaveis if var[0].isalpha()]

        # Inicializar um dicionário para armazenar as notas correspondentes a cada variável
        notas_aluno = {}

        # Iterar sobre as variáveis e buscar as notas correspondentes nas observações de avaliação
        for var in variaveis:
            nota = notas_disciplina[(notas_disciplina['aluno_id'] == aluno_id) & (notas_disciplina['aval'] == var)]['nota'].values
            if nota.size > 0:
                notas_aluno[var] = nota[0]

        # Substituir as variáveis na lógica pelas notas correspondentes e calcular a nota final
        try:
            # Substituir as variáveis na string de lógica pelo valor correspondente
            for var, val in notas_aluno.items():
                logica_nota_final = logica_nota_final.replace(var, str(val))
            nota_final = eval(logica_nota_final)
            
            # Adicionar os dados à tabela de notas finais por disciplina
            media_final_data.append({
                'disciplina_id': disciplina_id,
                'aluno_id': aluno_id,
                'nota_final': nota_final,
                'aprovado': nota_final >= media_final_minima
            })
        except:
            print(f'Não foi possível calcular a nota final para o aluno {aluno_id} na disciplina {disciplina_id}.')

# Criar DataFrame para a tabela de notas finais por disciplina
media_final_df = pd.DataFrame(media_final_data)


# Exemplo

In [ ]:
turma_df.loc[turma_df['aluno_id'] == 13]

In [ ]:
notas_df.loc[notas_df['aluno_id'] == 13]

In [ ]:
media_final_df.loc[media_final_df['aluno_id'] == 13]